In [1]:
import tweepy,json
access_token="2427460241-yihCbhCrkA6QrS7mhtwkK9FCnoKMvZzPNRFEtYr"
access_token_secret="p6dTQRMwy0SxVP49oHLUXeJ1L4T2cDqPM4RNlcEPErJ9X"
consumer_key="3nHkUhoqNif1x64w2gN7UxfPD"
consumer_secret="SD0qR7yWtUdGDqYxPcUpyNsRcMsX5MJB9z4MxjVKyq6VYwvDp0"
auth= tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth)

In [2]:
search_terms = ['home-cooked food for corona patients','home cooked food for corona patients','homemade food for corona patients','food for corona patients','food for covid patients', 'covid food delivery', 'covid free home cooked food delivery','homemade food covid','homemade food corona','home-cooked food covid','home-cooked food corona','home cooked food covid','home cooked food corona']
data = [] 
def stream_tweets(search_term):
    counter = 0 
    for tweet in tweepy.Cursor(api.search, q='\"{}\" -filter:retweets'.format(search_term), count=100, lang='en', tweet_mode='extended').items():
        tweet_details = {}
        tweet_details['time'] = str(tweet.created_at)
        tweet_details['id'] = tweet.id
        tweet_details['name'] = tweet.user.screen_name
        tweet_details['tweet'] = tweet.full_text
        tweet_details['retweets'] = tweet.retweet_count
        tweet_details['location'] = tweet.user.location
        tweet_details['created'] = tweet.created_at.strftime("%d-%b-%Y")
        tweet_details['followers'] = tweet.user.followers_count
        tweet_details['is_user_verified'] = tweet.user.verified
        data.append(tweet_details)
        counter += 1
        if counter == 1000:
            break
        else:
            pass
    with open('data.json', 'w') as f:
        json.dump(data, f)
    print('done!')

In [3]:
if __name__ == "__main__":
    print('Starting to stream...')
    for k in search_terms:
        stream_tweets(k)
    print('finished!')

Starting to stream...
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
finished!


In [4]:
#converting to csv for simplicity
import pandas as pd
df = pd.read_json('data.json')

df.to_csv('data_tweet.csv',index=None)

In [5]:
import pandas as pd
Data = pd.read_csv('data_tweet.csv',index_col=False)


Data1 = Data.where(pd.notnull(Data), None)

Data1.head()

,time,id,name,tweet,retweets,location,created,followers,is_user_verified
0,2021-05-03 09:32:46,1389150935792766979,DeepakT29274872,@SonuSood Free Food For \nCorona patients in N...,2,None,03-May-2021,2293,False
1,2021-05-03 09:30:54,1389150466303356928,DeepakT29274872,Free Food For \nCorona patients in Noida... ht...,1,None,03-May-2021,2293,False
2,2021-05-02 06:06:17,1388736583138889736,sv99999999,🙏\n#Covid19\n#Karnataka\n#Bengaluru \n#coronav...,1,India,02-May-2021,236,False
3,2021-04-30 07:00:47,1388025520823042055,Iamsainani,Free food for corona patients in vijayawada \n...,1,Khammam,30-Apr-2021,579,False
4,2021-04-28 15:58:33,1387436082208280579,Robojohn11,Ground Report - TamiNadu Govt Hosp's - Food wh...,1,"Chennai, India",28-Apr-2021,234,False


In [6]:
import spacy
from spacy import displacy 
# nlp = spacy.load('en_core_web_sm')

In [7]:
nlp_wk = spacy.load('xx_ent_wiki_sm')

In [8]:
import en_core_web_sm
nlp = en_core_web_sm.load()

In [9]:
import re
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\u2710-\u2900"
#         u"\u2757"
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

In [10]:
lower = lambda x: x.lower()
cap = lambda x: x.title()
bad_chars=['#','!','@']
replace=lambda x:[''.join(i for i in x if not i in bad_chars)]
remove_emoji=lambda x:deEmojify(x)

In [11]:
# df = pd.DataFrame(Data1['tweet'].apply(lower))
# df = pd.DataFrame(Data1['tweet'].apply(cap))
df = pd.DataFrame(Data1['tweet'].apply(replace))
df['tweet']=df['tweet'].str.get(0)
df = pd.DataFrame(df['tweet'].apply(remove_emoji))
df.columns = ['text']

In [12]:
df

,text
0,SonuSood Free Food For \nCorona patients in No...
1,Free Food For \nCorona patients in Noida... ht...
2,\nCovid19\nKarnataka\nBengaluru \ncoronavirus\...
3,Free food for corona patients in vijayawada \n...
4,Ground Report - TamiNadu Govt Hosp's - Food wh...
...,...
265,Food for Covid patients for free May try this ...
266,covidmealsforindia \n\nFree Food for Covid pat...
267,Someone has put this google doc with contact i...
268,If anybody wants home cooked food (COVID nutri...


In [13]:
def extract_named_ents(text):
    return [ent.text for ent in nlp(text).ents if ent.label_ == 'GPE']

def add_named_ents(df):
    df['named_ents'] = df['text'].apply(extract_named_ents)   

In [14]:
add_named_ents(df)

In [15]:
df['named_ents'] = df['named_ents'].str.get(0)
df = df.where(pd.notnull(df), "")

In [16]:
df

,text,named_ents
0,SonuSood Free Food For \nCorona patients in No...,Noida
1,Free Food For \nCorona patients in Noida... ht...,Noida
2,\nCovid19\nKarnataka\nBengaluru \ncoronavirus\...,
3,Free food for corona patients in vijayawada \n...,
4,Ground Report - TamiNadu Govt Hosp's - Food wh...,Kanyakumari
...,...,...
265,Food for Covid patients for free May try this ...,
266,covidmealsforindia \n\nFree Food for Covid pat...,
267,Someone has put this google doc with contact i...,
268,If anybody wants home cooked food (COVID nutri...,


In [17]:
df['named_ents']=df['named_ents'].apply(cap)
df

,text,named_ents
0,SonuSood Free Food For \nCorona patients in No...,Noida
1,Free Food For \nCorona patients in Noida... ht...,Noida
2,\nCovid19\nKarnataka\nBengaluru \ncoronavirus\...,
3,Free food for corona patients in vijayawada \n...,
4,Ground Report - TamiNadu Govt Hosp's - Food wh...,Kanyakumari
...,...,...
265,Food for Covid patients for free May try this ...,
266,covidmealsforindia \n\nFree Food for Covid pat...,
267,Someone has put this google doc with contact i...,
268,If anybody wants home cooked food (COVID nutri...,


In [18]:
urls_list=[]
for tweet in Data['tweet']:
    urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', tweet)
    st =  ",".join(urls)
    urls_list.append(st)
    for url in urls:
        try:
            res = urllib2.urlopen(url)
            actual_url = res.geturl()
            print(actual_url)
        except:
            print(url)

https://t.co/kLEfjEnCuc
https://t.co/hh7CSq4dkl
https://t.co/hRZfxKK2XU
https://t.co/uP4KbY2TLP
https://t.co/GbxzFiJEPv
https://t.co/GhKwNTRmC2
https://t.co/22UrhyxTdk
https://t.co/sm4Cf3ZQ4v
https://t.co/oBEcdNBBwN
https://t.co/vaYL3cLbgP
https://t.co/JUAOSE2KQ5
https://t.co/hfiWQ1lekR
https://t.co/bfH3uuDVlr
https://t.co/6Ko7Evkn8k
https://t.co/NbjEXlBVMh
https://t.co/A1hFuO8iJ3
https://t.co/fGqwlg1YFa
https://t.co/lGGE87sYOb
https://t.co/6QU6SbMc5U
https://t.co/6YofQ2nUKO
https://t.co/c6EuJEw6yk
https://t.co/76IRFEP7bB
https://t.co/EcayCJiEoa
https://t.co/KeF1z4jwa6
https://t.co/NifpEvUttD
https://t.co/Mn4czQI8Ns
https://t.co/aKB4SPIsQL
https://t.co/qyfvowX7I9
https://t.co/wg40qOJztH
https://t.co/4N2ei68D39
https://t.co/ipQo8d1ZMg
https://t.co/xb8BiNUHAy
https://t.co/2YzRweUPob
https://t.co/2YzRweUPob
https://t.co/2YzRweUPob
https://t.co/ZEAD4IQ6q5
https://t.co/XB5MnZO0pl
https://t.co/NyLze1zVFz
https://t.co/itWgiZ1uMT
https://t.co/e2niBUxars
https://t.co/auDPVM7us9
https://t.co/tkQ

In [19]:
Data1['Urls'] = urls_list

In [20]:
for urls in Data1['Urls']:
    print(urls)
#     print('b')

https://t.co/kLEfjEnCuc
https://t.co/hh7CSq4dkl
https://t.co/hRZfxKK2XU
https://t.co/uP4KbY2TLP

https://t.co/GbxzFiJEPv
https://t.co/GhKwNTRmC2
https://t.co/22UrhyxTdk
https://t.co/sm4Cf3ZQ4v
https://t.co/oBEcdNBBwN
https://t.co/vaYL3cLbgP
https://t.co/JUAOSE2KQ5
https://t.co/hfiWQ1lekR
https://t.co/bfH3uuDVlr
https://t.co/6Ko7Evkn8k
https://t.co/NbjEXlBVMh

https://t.co/A1hFuO8iJ3

https://t.co/fGqwlg1YFa


https://t.co/lGGE87sYOb,https://t.co/6QU6SbMc5U
https://t.co/6YofQ2nUKO
https://t.co/c6EuJEw6yk
https://t.co/76IRFEP7bB
https://t.co/EcayCJiEoa
https://t.co/KeF1z4jwa6

https://t.co/NifpEvUttD
https://t.co/Mn4czQI8Ns

https://t.co/aKB4SPIsQL
https://t.co/qyfvowX7I9
https://t.co/wg40qOJztH



https://t.co/4N2ei68D39
https://t.co/ipQo8d1ZMg,https://t.co/xb8BiNUHAy

https://t.co/2YzRweUPob
https://t.co/2YzRweUPob
https://t.co/2YzRweUPob



https://t.co/ZEAD4IQ6q5


https://t.co/XB5MnZO0pl


https://t.co/NyLze1zVFz,https://t.co/itWgiZ1uMT
https://t.co/e2niBUxars



https://t.co/auDPVM

In [21]:
Data1

,time,id,name,tweet,retweets,location,created,followers,is_user_verified,Urls
0,2021-05-03 09:32:46,1389150935792766979,DeepakT29274872,@SonuSood Free Food For \nCorona patients in N...,2,None,03-May-2021,2293,False,https://t.co/kLEfjEnCuc
1,2021-05-03 09:30:54,1389150466303356928,DeepakT29274872,Free Food For \nCorona patients in Noida... ht...,1,None,03-May-2021,2293,False,https://t.co/hh7CSq4dkl
2,2021-05-02 06:06:17,1388736583138889736,sv99999999,🙏\n#Covid19\n#Karnataka\n#Bengaluru \n#coronav...,1,India,02-May-2021,236,False,https://t.co/hRZfxKK2XU
3,2021-04-30 07:00:47,1388025520823042055,Iamsainani,Free food for corona patients in vijayawada \n...,1,Khammam,30-Apr-2021,579,False,https://t.co/uP4KbY2TLP
4,2021-04-28 15:58:33,1387436082208280579,Robojohn11,Ground Report - TamiNadu Govt Hosp's - Food wh...,1,"Chennai, India",28-Apr-2021,234,False,
...,...,...,...,...,...,...,...,...,...,...
265,2021-04-26 02:15:51,1386504264965517316,_rakeshranjan__,Food for Covid patients for free! May try this...,0,None,26-Apr-2021,2,False,https://t.co/PFSomY4py8
266,2021-04-26 00:04:30,1386471211991912448,InvestBooks,#covidmealsforindia \n\nFree Food for Covid pa...,2,India,26-Apr-2021,30485,False,https://t.co/Uytmv5rYHx
267,2021-04-26 14:12:27,1386684602102665222,thecommonman__,Someone has put this google doc with contact i...,50,யாதும் ஊரே...யாவரும் கேளிர்,26-Apr-2021,70263,False,https://t.co/FgY4wpdauK
268,2021-04-26 06:21:30,1386566085474807814,nupoorvibhandik,If anybody wants home cooked food (COVID nutri...,2,"Nashik, India",26-Apr-2021,4,False,https://t.co/nwVgZHQiFO


In [22]:
arr=[
"Andaman and Nicobar Islands"
,"Port Blair"
,"Andhra Pradesh"
,"Adoni"
,"Amaravati"
,"Anantapur"
,"Chandragiri"
,"Chittoor"
,"Dowlaiswaram"
,"Eluru"
,"Guntur"
,"Kadapa"
,"Kakinada"
,"Kurnool"
,"Machilipatnam"
,"Nagarjunakoṇḍa"
,"Rajahmundry"
,"Srikakulam"
,"Tirupati"
,"Vijayawada"
,"Visakhapatnam"
,"Vizianagaram"
,"Yemmiganur"
,"Arunachal Pradesh"
,"Itanagar"
,"Assam"
,"Dhuburi"
,"Dibrugarh"
,"Dispur"
,"Guwahati"
,"Jorhat"
,"Nagaon"
,"Sibsagar"
,"Silchar"
,"Tezpur"
,"Tinsukia"
,"Bihar"
,"Ara"
,"Baruni"
,"Begusarai"
,"Bettiah"
,"Bhagalpur"
,"Bihar Sharif"
,"Bodh Gaya"
,"Buxar"
,"Chapra"
,"Darbhanga"
,"Dehri"
,"Dinapur Nizamat"
,"Gaya"
,"Hajipur"
,"Jamalpur"
,"Katihar"
,"Madhubani"
,"Motihari"
,"Munger"
,"Muzaffarpur"
,"Patna"
,"Purnia"
,"Pusa"
,"Saharsa"
,"Samastipur"
,"Sasaram"
,"Sitamarhi"
,"Siwan"
,"Chandigarh"
,"Chandigarh"
,"Chhattisgarh"
,"Ambikapur"
,"Bhilai"
,"Bilaspur"
,"Dhamtari"
,"Durg"
,"Jagdalpur"
,"Raipur"
,"Rajnandgaon"
,"Dadra and Nagar Haveli"
,"Silvassa"
,"Daman and Diu"
,"Daman"
,"Diu"
,"Delhi"
,"Delhi"
,"New Delhi"
,"Goa"
,"Madgaon"
,"Panaji"
,"Gujarat"
,"Ahmadabad"
,"Amreli"
,"Bharuch"
,"Bhavnagar"
,"Bhuj"
,"Dwarka"
,"Gandhinagar"
,"Godhra"
,"Jamnagar"
,"Junagadh"
,"Kandla"
,"Khambhat"
,"Kheda"
,"Mahesana"
,"Morvi"
,"Nadiad"
,"Navsari"
,"Okha"
,"Palanpur"
,"Patan"
,"Porbandar"
,"Rajkot"
,"Surat"
,"Surendranagar"
,"Valsad"
,"Veraval"
,"Haryana"
,"Ambala"
,"Bhiwani"
,"Chandigarh"
,"Faridabad"
,"Firozpur Jhirka"
,"Gurgaon"
,"Hansi"
,"Hisar"
,"Jind"
,"Kaithal"
,"Karnal"
,"Kurukshetra"
,"Panipat"
,"Pehowa"
,"Rewari"
,"Rohtak"
,"Sirsa"
,"Sonipat"
,"Himachal Pradesh"
,"Bilaspur"
,"Chamba"
,"Dalhousie"
,"Dharmshala"
,"Hamirpur"
,"Kangra"
,"Kullu"
,"Mandi"
,"Nahan"
,"Shimla"
,"Una"
,"Jammu"
,"Kashmir"
,"Anantnag"
,"Baramula"
,"Doda"
,"Gulmarg"
,"Jammu"
,"Kathua"
,"Punch"
,"Rajauri"
,"Srinagar"
,"Udhampur"
,"Jharkhand"
,"Bokaro"
,"Chaibasa"
,"Deoghar"
,"Dhanbad"
,"Dumka"
,"Giridih"
,"Hazaribag"
,"Jamshedpur"
,"Jharia"
,"Rajmahal"
,"Ranchi"
,"Saraikela"
,"Karnataka"
,"Badami"
,"Ballari"
,"Bangalore"
,"Bangaluru"
,"Bengaluru"
,"Belgavi"
,"Bhadravati"
,"Bidar"
,"Chikkamagaluru"
,"Chitradurga"
,"Davangere"
,"Halebid"
,"Hassan"
,"Hubballi-Dharwad"
,"Kalaburagi"
,"Kolar"
,"Madikeri"
,"Mandya"
,"Mangaluru"
,"Mysuru"
,"Raichur"
,"Shivamogga"
,"Shravanabelagola"
,"Shrirangapattana"
,"Tumkuru"
,"Kerala"
,"Alappuzha"
,"Badagara"
,"Idukki"
,"Kannur"
,"Kochi"
,"Kollam"
,"Kottayam"
,"Kozhikode"
,"Mattancheri"
,"Palakkad"
,"Thalassery"
,"Thiruvananthapuram"
,"Thrissur"
,"Ladakh "
,"Kargil"
,"Leh"
,"Madhya Pradesh"
,"Balaghat"
,"Barwani"
,"Betul"
,"Bharhut"
,"Bhind"
,"Bhojpur"
,"Bhopal"
,"Burhanpur"
,"Chhatarpur"
,"Chhindwara"
,"Damoh"
,"Datia"
,"Dewas"
,"Dhar"
,"Guna"
,"Gwalior"
,"Hoshangabad"
,"Indore"
,"Itarsi"
,"Jabalpur"
,"Jhabua"
,"Khajuraho"
,"Khandwa"
,"Khargon"
,"Maheshwar"
,"Mandla"
,"Mandsaur"
,"Mhow"
,"Morena"
,"Murwara"
,"Narsimhapur"
,"Narsinghgarh"
,"Narwar"
,"Neemuch"
,"Nowgong"
,"Orchha"
,"Panna"
,"Raisen"
,"Rajgarh"
,"Ratlam"
,"Rewa"
,"Sagar"
,"Sarangpur"
,"Satna"
,"Sehore"
,"Seoni"
,"Shahdol"
,"Shajapur"
,"Sheopur"
,"Shivpuri"
,"Ujjain"
,"Vidisha"
,"Maharashtra"
,"Ahmadnagar"
,"Akola"
,"Amravati"
,"Aurangabad"
,"Bhandara"
,"Bhusawal"
,"Bid"
,"Buldana"
,"Chandrapur"
,"Daulatabad"
,"Dhule"
,"Jalgaon"
,"Kalyan"
,"Karli"
,"Kolhapur"
,"Mahabaleshwar"
,"Malegaon"
,"Matheran"
,"Mumbai"
,"Nagpur"
,"Nanded"
,"Nashik"
,"Osmanabad"
,"Pandharpur"
,"Parbhani"
,"Pune"
,"Ratnagiri"
,"Sangli"
,"Satara"
,"Sevagram"
,"Solapur"
,"Thane"
,"Ulhasnagar"
,"Vasai-Virar"
,"Wardha"
,"Yavatmal"
,"Manipur"
,"Imphal"
,"Meghalaya"
,"Cherrapunji"
,"Shillong"
,"Mizoram"
,"Aizawl"
,"Lunglei"
,"Nagaland"
,"Kohima"
,"Mon"
,"Phek"
,"Wokha"
,"Zunheboto"
,"Odisha"
,"Balangir"
,"Baleshwar"
,"Baripada"
,"Bhubaneshwar"
,"Brahmapur"
,"Cuttack"
,"Dhenkanal"
,"Keonjhar"
,"Konark"
,"Koraput"
,"Paradip"
,"Phulabani"
,"Puri"
,"Sambalpur"
,"Udayagiri"
,"Puducherry"
,"Karaikal"
,"Mahe"
,"Puducherry"
,"Yanam"
,"Punjab"
,"Amritsar"
,"Batala"
,"Chandigarh"
,"Faridkot"
,"Firozpur"
,"Gurdaspur"
,"Hoshiarpur"
,"Jalandhar"
,"Kapurthala"
,"Ludhiana"
,"Nabha"
,"Patiala"
,"Rupnagar"
,"Sangrur"
,"Rajasthan"
,"Abu"
,"Ajmer"
,"Alwar"
,"Amer"
,"Barmer"
,"Beawar"
,"Bharatpur"
,"Bhilwara"
,"Bikaner"
,"Bundi"
,"Chittaurgarh"
,"Churu"
,"Dhaulpur"
,"Dungarpur"
,"Ganganagar"
,"Hanumangarh"
,"Jaipur"
,"Jaisalmer"
,"Jalor"
,"Jhalawar"
,"Jhunjhunu"
,"Jodhpur"
,"Kishangarh"
,"Kota"
,"Merta"
,"Nagaur"
,"Nathdwara"
,"Pali"
,"Phalodi"
,"Pushkar"
,"Sawai Madhopur"
,"Shahpura"
,"Sikar"
,"Sirohi"
,"Tonk"
,"Udaipur"
,"Sikkim"
,"Gangtok"
,"Gyalsing"
,"Lachung"
,"Mangan"
,"Tamil Nadu"
,"Arcot"
,"Chengalpattu"
,"Chennai"
,"Chidambaram"
,"Coimbatore"
,"Cuddalore"
,"Dharmapuri"
,"Dindigul"
,"Erode"
,"Kanchipuram"
,"Kanniyakumari"
,"Kodaikanal"
,"Kumbakonam"
,"Madurai"
,"Mamallapuram"
,"Nagappattinam"
,"Nagercoil"
,"Palayankottai"
,"Pudukkottai"
,"Rajapalaiyam"
,"Ramanathapuram"
,"Salem"
,"Thanjavur"
,"Tiruchchirappalli"
,"Tirunelveli"
,"Tiruppur"
,"Tuticorin"
,"Udhagamandalam"
,"Vellore"
,"Telangana"
,"Hyderabad"
,"Karimnagar"
,"Khammam"
,"Mahbubnagar"
,"Nizamabad"
,"Sangareddi"
,"Warangal"
,"Tripura"
,"Agartala"
,"Uttar Pradesh"
,"Agra"
,"Aligarh"
,"Amroha"
,"Ayodhya"
,"Azamgarh"
,"Bahraich"
,"Ballia"
,"Banda"
,"Bara Banki"
,"Bareilly"
,"Basti"
,"Bijnor"
,"Bithur"
,"Budaun"
,"Bulandshahr"
,"Deoria"
,"Etah"
,"Etawah"
,"Faizabad"
,"Farrukhabad"
,"Fatehgarh"
,"Fatehpur"
,"Fatehpur Sikri"
,"Ghaziabad"
,"Ghazipur"
,"Gonda"
,"Gorakhpur"
,"Hamirpur"
,"Hardoi"
,"Hathras"
,"Jalaun"
,"Jaunpur"
,"Jhansi"
,"Kannauj"
,"Kanpur"
,"Lakhimpur"
,"Lalitpur"
,"Lucknow"
,"Mainpuri"
,"Mathura"
,"Meerut"
,"Mirzapur"
,"Vindhyachal"
,"Moradabad"
,"Muzaffarnagar"
,"Partapgarh"
,"Pilibhit"
,"Prayagraj"
,"Rae Bareli"
,"Rampur"
,"Saharanpur"
,"Sambhal"
,"Shahjahanpur"
,"Sitapur"
,"Sultanpur"
,"Tehri"
,"Varanasi"
,"Uttarakhand"
,"Almora"
,"Dehra Dun"
,"Haridwar"
,"Mussoorie"
,"Nainital"
,"Pithoragarh"
,"West Bengal"
,"Alipore"
,"Alipur Duar"
,"Asansol"
,"Baharampur"
,"Bally"
,"Balurghat"
,"Bankura"
,"Baranagar"
,"Barasat"
,"Barrackpore"
,"Basirhat"
,"Bhatpara"
,"Bishnupur"
,"Budge Budge"
,"Burdwan"
,"Chandernagore"
,"Darjiling"
,"Diamond Harbour"
,"Dum Dum"
,"Durgapur"
,"Halisahar"
,"Haora"
,"Hugli"
,"Ingraj Bazar"
,"Jalpaiguri"
,"Kalimpong"
,"Kamarhati"
,"Kanchrapara"
,"Kharagpur"
,"Koch Bihar"
,"Kolkata"
,"Krishnanagar"
,"Malda"
,"Midnapore"
,"Murshidabad"
,"Navadwip"
,"Palashi"
,"Panihati"
,"Purulia"
,"Raiganj"
,"Santipur"
,"Shantiniketan"
,"Shrirampur"
,"Siliguri"
,"Siuri"
,"Tamluk"
]

In [23]:
df['text']=df['text'].apply(cap)
for i in range(0,len(df)):
    if df['named_ents'][i]=="":
        for k in arr:
            if k in df['text'][i]:
                df['named_ents'][i]=k
    elif df['named_ents'][i]!=None:
        for k in arr:
            if k in df['text'][i]and k not in df['named_ents'][i]:
                df['named_ents'][i]+=','+k
        

In [24]:
df

,text,named_ents
0,Sonusood Free Food For \nCorona Patients In No...,Noida
1,Free Food For \nCorona Patients In Noida... Ht...,Noida
2,\nCovid19\nKarnataka\nBengaluru \nCoronavirus\...,Bengaluru
3,Free Food For Corona Patients In Vijayawada \n...,Vijayawada
4,Ground Report - Taminadu Govt Hosp'S - Food Wh...,Kanyakumari
...,...,...
265,Food For Covid Patients For Free May Try This ...,
266,Covidmealsforindia \n\nFree Food For Covid Pat...,
267,Someone Has Put This Google Doc With Contact I...,Coimbatore
268,If Anybody Wants Home Cooked Food (Covid Nutri...,Nashik


In [25]:
Data1['tweet_location']=df['named_ents']

In [26]:
import numpy as np
def explode(df, lst_cols, fill_value='', preserve_index=False):
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0
        and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    return res

In [27]:
Data1=explode(Data1.assign(tweet_location=Data1.tweet_location.str.split(',')), 'tweet_location')

In [28]:
dictionary={'Bengaluru':'Bangalore','Bangaluru':'Bangalore','Hsr':'Bangalore',
           'District Faridabad':'Faridabad',
           'North Delhi':'Delhi','West Delhi':'Delhi','East Delhi':'Delhi','South Delhi':'Delhi','New Delhi':'Delhi','Old Delhi':'Delhi'}

for i in range(0,len(Data1)):
    for k,v in dictionary.items():
        if Data1['tweet_location'][i]==k:
            Data1['tweet_location'][i]=v


C:\Users\Khushboo\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [29]:
Data1

,Urls,created,followers,id,is_user_verified,location,name,retweets,time,tweet,tweet_location
0,https://t.co/kLEfjEnCuc,03-May-2021,2293,1389150935792766979,False,None,DeepakT29274872,2,2021-05-03 09:32:46,@SonuSood Free Food For \nCorona patients in N...,Noida
1,https://t.co/hh7CSq4dkl,03-May-2021,2293,1389150466303356928,False,None,DeepakT29274872,1,2021-05-03 09:30:54,Free Food For \nCorona patients in Noida... ht...,Noida
2,https://t.co/hRZfxKK2XU,02-May-2021,236,1388736583138889736,False,India,sv99999999,1,2021-05-02 06:06:17,🙏\n#Covid19\n#Karnataka\n#Bengaluru \n#coronav...,Bangalore
3,https://t.co/uP4KbY2TLP,30-Apr-2021,579,1388025520823042055,False,Khammam,Iamsainani,1,2021-04-30 07:00:47,Free food for corona patients in vijayawada \n...,Vijayawada
4,,28-Apr-2021,234,1387436082208280579,False,"Chennai, India",Robojohn11,1,2021-04-28 15:58:33,Ground Report - TamiNadu Govt Hosp's - Food wh...,Kanyakumari
...,...,...,...,...,...,...,...,...,...,...,...
293,https://t.co/PFSomY4py8,26-Apr-2021,2,1386504264965517316,False,None,_rakeshranjan__,0,2021-04-26 02:15:51,Food for Covid patients for free! May try this...,
294,https://t.co/Uytmv5rYHx,26-Apr-2021,30485,1386471211991912448,False,India,InvestBooks,2,2021-04-26 00:04:30,#covidmealsforindia \n\nFree Food for Covid pa...,
295,https://t.co/FgY4wpdauK,26-Apr-2021,70263,1386684602102665222,False,யாதும் ஊரே...யாவரும் கேளிர்,thecommonman__,50,2021-04-26 14:12:27,Someone has put this google doc with contact i...,Coimbatore
296,https://t.co/nwVgZHQiFO,26-Apr-2021,4,1386566085474807814,False,"Nashik, India",nupoorvibhandik,2,2021-04-26 06:21:30,If anybody wants home cooked food (COVID nutri...,Nashik


In [30]:
Data1=Data1[['time','id','name','tweet','retweets','location','created','followers','is_user_verified','Urls','tweet_location']]

In [31]:
Data1

,time,id,name,tweet,retweets,location,created,followers,is_user_verified,Urls,tweet_location
0,2021-05-03 09:32:46,1389150935792766979,DeepakT29274872,@SonuSood Free Food For \nCorona patients in N...,2,None,03-May-2021,2293,False,https://t.co/kLEfjEnCuc,Noida
1,2021-05-03 09:30:54,1389150466303356928,DeepakT29274872,Free Food For \nCorona patients in Noida... ht...,1,None,03-May-2021,2293,False,https://t.co/hh7CSq4dkl,Noida
2,2021-05-02 06:06:17,1388736583138889736,sv99999999,🙏\n#Covid19\n#Karnataka\n#Bengaluru \n#coronav...,1,India,02-May-2021,236,False,https://t.co/hRZfxKK2XU,Bangalore
3,2021-04-30 07:00:47,1388025520823042055,Iamsainani,Free food for corona patients in vijayawada \n...,1,Khammam,30-Apr-2021,579,False,https://t.co/uP4KbY2TLP,Vijayawada
4,2021-04-28 15:58:33,1387436082208280579,Robojohn11,Ground Report - TamiNadu Govt Hosp's - Food wh...,1,"Chennai, India",28-Apr-2021,234,False,,Kanyakumari
...,...,...,...,...,...,...,...,...,...,...,...
293,2021-04-26 02:15:51,1386504264965517316,_rakeshranjan__,Food for Covid patients for free! May try this...,0,None,26-Apr-2021,2,False,https://t.co/PFSomY4py8,
294,2021-04-26 00:04:30,1386471211991912448,InvestBooks,#covidmealsforindia \n\nFree Food for Covid pa...,2,India,26-Apr-2021,30485,False,https://t.co/Uytmv5rYHx,
295,2021-04-26 14:12:27,1386684602102665222,thecommonman__,Someone has put this google doc with contact i...,50,யாதும் ஊரே...யாவரும் கேளிர்,26-Apr-2021,70263,False,https://t.co/FgY4wpdauK,Coimbatore
296,2021-04-26 06:21:30,1386566085474807814,nupoorvibhandik,If anybody wants home cooked food (COVID nutri...,2,"Nashik, India",26-Apr-2021,4,False,https://t.co/nwVgZHQiFO,Nashik


In [36]:
# export_excel = Data1.to_excel (r'C:\Users\Khushboo\Desktop\Cargill\sample.xlsx', index = None, header=True)

In [37]:
# export_excel = df.to_excel (r'C:\Users\Khushboo\Desktop\Cargill\sample1.xlsx', index = None, header=True)

In [34]:
#creating connection to mysql and creating db
!pip3 install mysql.connector
import mysql.connector as msql
from mysql.connector import Error
try:
    conn = msql.connect(host='sql129.main-hosting.eu', user='u291509283_cargill',  
                        password='Cargill123')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("CREATE DATABASE u291509283_cargill")
        print("Tweet_DB database is created")
except Error as e:
    print("Error while connecting to MySQL", e)

  Using cached https://files.pythonhosted.org/packages/28/04/e40098f3730e75bbe36a338926f566ea803550a34fb50535499f4fc4787a/mysql-connector-2.2.9.tar.gz
  Running setup.py install for mysql.connector: started
    Running setup.py install for mysql.connector: finished with status 'done'


You are using pip version 19.0.3, however version 21.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


Error while connecting to MySQL 1007 (HY000): Can't create database 'u291509283_cargill'; database exists


In [35]:
#updation in database
try:
    conn = msql.connect(host='sql129.main-hosting.eu', 
                           database='u291509283_cargill', user='u291509283_cargill', 
                           password='Cargill123')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        cursor.execute('DROP TABLE IF EXISTS Tweet_data;')
        print('Creating table....')
        cursor.execute("CREATE TABLE Tweet_data (time DATETIME,user_id BIGINT,name VARCHAR(255), tweet VARCHAR(255) ,retweets INTEGER,location VARCHAR(255),created VARCHAR(255),followers INTEGER,is_user_verified VARCHAR(255),Urls VARCHAR(255), Tweet_location VARCHAR(255))")
        print("Tweet table is created....")
        for i,row in Data1.iterrows():
            sql = "INSERT INTO u291509283_cargill.Tweet_data VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row))
            print("Record inserted")
            
            conn.commit()
        
except Error as e:
    print("Error while connecting to MySQL", e)

You're connected to database:  ('u291509283_cargill',)
Creating table....
Tweet table is created....
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Rec

In [227]:
excel_loc = r'C:\Users\user\Downloads\Food Providers List.xlsx'

In [228]:
excel_data = pd.read_excel(excel_loc, sheet_name='Food Providers across India',header =1)

In [229]:
excel_data

,Sl. No.,Name of the Supplier*,Phone Number*,Location,City*,State,Cost (if any),Added By*,Verified,Notes,Verification to be done by,Comments,Include
0,1.0,Abhinandan,9606107374,Kumaraswamy layout,Bangalore,Karnataka,NaN,Aravinda Shenoy,Y,Validated and supplies provided,Hema,NaN,Y
1,2.0,Aduggemane,"7892403742, 9916435389",South Bangalore,Bangalore,Karnataka,NaN,Aravinda Shenoy,Y,1st Number: Ringing but no one picked,Hema,NaN,N
2,3.0,Aishwarya S + Upendra,9742999549,New thippasandra/Indiranagar/CV Raman nagar,Bangalore,Karnataka,NaN,Aravinda Shenoy,Y,2nd Number: Not reachable,Hema,NaN,N
3,4.0,Akhila Varghese,"080-41314586, 7013510980","Whitefield, Hope Farm, Kadugodi, chanasandhra,...",Bangalore,Karnataka,NaN,Aravinda Shenoy,Y,Validated and supplies provided,Hema,NaN,Y
4,5.0,Akshay,9964301555,Sahakarnagar,Bangalore,Karnataka,NaN,Aravinda Shenoy,Y,Validated and supplies provided,Hema,NaN,Y
5,6.0,Amrutha(Amrutha mess),"9945374603, 9916602399",Tavarekere Main road,Bangalore,Karnataka,NaN,Aravinda Shenoy,Y,*orders have to be placed 24 hours before,Hema,NaN,Y
6,7.0,Anagha,7899942729,Whitefield,Bangalore,Karnataka,NaN,Aravinda Shenoy,Y,Not reachable,Hema,NaN,N
7,8.0,Anamika,7760341030,Koramangala,Bangalore,Karnataka,NaN,Aravinda Shenoy,Y,She took her vaccines recently and not keeping...,Hema,https://www.gordian.in/ ; Until 4/May,N
8,9.0,Aneesha,8861690548,Doddakanehalli,Bangalore,Karnataka,NaN,Aravinda Shenoy,Y,Validated and supplies provided,Hema,NaN,Y
9,10.0,Anisha Padhee,9833366152,"Kanakapura Road, Konankunte Cross, Kumaraswamy...",Bangalore,Karnataka,NaN,Aravinda Shenoy,Y,Validated and supplies provided,Hema,NaN,Y


In [230]:
excel_data['City*']=excel_data['City*'].str.strip().fillna('')
excel_data['Location']=excel_data['Location'].str.strip().fillna('')
excel_data['Verified']=excel_data['Verified'].fillna('N')

In [231]:
db_mapper = pd.DataFrame(columns = ['time','user_id','name','tweet','tweet_location','tweet_type','verified','notes'])

In [232]:
db_mapper

,time,user_id,name,tweet,tweet_location,tweet_type,verified,notes


In [236]:
import datetime
for index,rows in excel_data.iterrows():
    db_mapper.loc[index,'time']=datetime.datetime.now()
    db_mapper.loc[index,'user_id']=10000 + index
    db_mapper.loc[index,'name']='Cargill_CMT'
    db_mapper.loc[index,'tweet_location']=excel_data.loc[index,'City*']
    db_mapper.loc[index,'tweet_type']='cmt_volunteers'
    db_mapper.loc[index,'verified']=excel_data.loc[index,'Verified']
    db_mapper.loc[index,'notes']=excel_data.loc[index,'Notes']

    
db_mapper['tweet']= 'Name of the Supplier*:' + excel_data['Name of the Supplier*'] +" " + '<br>Phone Number*:' + excel_data['Phone Number*'].astype(str)+ " " + '<br>Location:'+ excel_data['Location'] + " " + '<br>City:'+ excel_data['City*'] + " " + '<br>Verified:'+ excel_data['Verified']   
db_mapper['notes']=db_mapper['notes'].fillna('')
db_mapper['tweet_location']=db_mapper['tweet_location'].fillna('')
db_mapper['verified']=db_mapper['verified'].fillna('')

In [237]:
db_mapper

,time,user_id,name,tweet,tweet_location,tweet_type,verified,notes
0,2021-05-06 00:49:14.464568,10000,Cargill_CMT,Name of the Supplier*:Abhinandan <br>Phone Num...,Bangalore,cmt_volunteers,Y,Validated and supplies provided
1,2021-05-06 00:49:14.471562,10001,Cargill_CMT,Name of the Supplier*:Aduggemane <br>Phone Num...,Bangalore,cmt_volunteers,Y,1st Number: Ringing but no one picked
2,2021-05-06 00:49:14.476562,10002,Cargill_CMT,Name of the Supplier*:Aishwarya S + Upendra <b...,Bangalore,cmt_volunteers,Y,2nd Number: Not reachable
3,2021-05-06 00:49:14.482557,10003,Cargill_CMT,Name of the Supplier*:Akhila Varghese <br>Phon...,Bangalore,cmt_volunteers,Y,Validated and supplies provided
4,2021-05-06 00:49:14.489554,10004,Cargill_CMT,Name of the Supplier*:Akshay <br>Phone Number*...,Bangalore,cmt_volunteers,Y,Validated and supplies provided
5,2021-05-06 00:49:14.494550,10005,Cargill_CMT,Name of the Supplier*:Amrutha(Amrutha mess) <b...,Bangalore,cmt_volunteers,Y,*orders have to be placed 24 hours before
6,2021-05-06 00:49:14.499551,10006,Cargill_CMT,Name of the Supplier*:Anagha <br>Phone Number*...,Bangalore,cmt_volunteers,Y,Not reachable
7,2021-05-06 00:49:14.503546,10007,Cargill_CMT,Name of the Supplier*:Anamika <br>Phone Number...,Bangalore,cmt_volunteers,Y,She took her vaccines recently and not keeping...
8,2021-05-06 00:49:14.507544,10008,Cargill_CMT,Name of the Supplier*:Aneesha <br>Phone Number...,Bangalore,cmt_volunteers,Y,Validated and supplies provided
9,2021-05-06 00:49:14.511545,10009,Cargill_CMT,Name of the Supplier*:Anisha Padhee <br>Phone ...,Bangalore,cmt_volunteers,Y,Validated and supplies provided


In [238]:
!pip install mysql.connector
import mysql.connector as msql
from mysql.connector import Error
try:
    conn = msql.connect(host='sql129.main-hosting.eu', 
                           database='u291509283_cargill', user='u291509283_cargill', 
                           password='Cargill123')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        del_sql = "DELETE FROM `Tweet_data` WHERE name='Cargill_CMT'"
        cursor.execute(del_sql)
        conn.commit()
        print("Delete share-point data")
        for index, row in db_mapper.iterrows():
            sql = "INSERT INTO u291509283_cargill.Tweet_data (time,user_id,name,tweet,tweet_location,tweet_type,verified,notes) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row))
            print("Record inserted")
            conn.commit()
        
except Error as e:
    print("Error while connecting to MySQL", e)
            


You're connected to database:  ('u291509283_cargill',)
Delete share-point data
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record in